## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data

columns = attrition_df[['Education', 'EnvironmentSatisfaction', 'JobLevel', 'JobSatisfaction', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion']]
# Create X_df using your selected columns

X_df = columns

# Show the data types for X_df

X_df.dtypes

Education                   int64
EnvironmentSatisfaction     int64
JobLevel                    int64
JobSatisfaction             int64
PerformanceRating           int64
RelationshipSatisfaction    int64
StockOptionLevel            int64
TrainingTimesLastYear       int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(X_df, y_df, test_size=0.2, random_state=15)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

#all columns are already numeric no need to convert 

In [8]:
# Create a StandardScaler

scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit_transform(train_x)

# Scale the training and testing data

x_trained_scaled = scaler.fit_transform(train_x)
x_test_scaled = scaler.transform(test_x)


In [9]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column

department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data

department_encoder.fit(train_y[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data

train_y_encoded = department_encoder.fit_transform(train_y[['Department']])
test_y_encoded = department_encoder.transform(test_y[['Department']])


In [10]:
from sklearn.preprocessing import OneHotEncoder


In [11]:
# Create a OneHotEncoder for the Attrition column

attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data

attrition_encoder.fit(train_y[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_encoded = attrition_encoder.fit_transform(train_y[['Attrition']])
y_test_encoded = attrition_encoder.transform(test_y[['Attrition']])



## Part 2: Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data.

input_dim = x_trained_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [13]:
# Create a branch for Department
# with a hidden layer and an output layer
# Create the hidden layer

department_hidden_layer = layers.Dense(16, activation="relu")(shared_layer2)

# Create the output layer

dept_output_layer = layers.Dense(train_y_encoded.shape[1], activation= 'softmax', name='dept_output')(department_hidden_layer)

In [14]:
# Create a branch for Attrition
# Create the hidden layer

attrition_hidden_layer = layers.Dense(16, activation="relu")(shared_layer2)

# Create the output layer

attrition_output_layer = layers.Dense(y_train_encoded.shape[1], activation= 'softmax', name='attrition_output')(attrition_hidden_layer)

In [21]:
# Create the model

da_model = Model(inputs= input_layer, outputs= [dept_output_layer, attrition_output_layer])

# Compile the model
da_model.compile (optimizer ='adam',
                 loss ={'dept_output': 'categorical_crossentropy','attrition_output': 'categorical_crossentropy'},
                 metrics ={'dept_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model

da_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        768 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         51 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,989 (15.58 KB)

 Trainable params: 3,989 (15.58 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Train the model
trained_da_mdoel = da_model.fit(x_trained_scaled,
                                {'dept_output': train_y_encoded,
                                'attrition_output': y_train_encoded},
                                epochs=50,
                                batch_size = 25,
                                validation_data = (x_test_scaled, {'dept_output': test_y_encoded,
                                'attrition_output': y_test_encoded}),
                                verbose = 1)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8247 - attrition_output_loss: 0.4420 - dept_output_accuracy: 0.6430 - dept_output_loss: 0.8392 - loss: 1.2817 - val_attrition_output_accuracy: 0.8605 - val_attrition_output_loss: 0.4180 - val_dept_output_accuracy: 0.6803 - val_dept_output_loss: 0.7732 - val_loss: 1.1915
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8140 - attrition_output_loss: 0.4335 - dept_output_accuracy: 0.6390 - dept_output_loss: 0.7876 - loss: 1.2215 - val_attrition_output_accuracy: 0.8605 - val_attrition_output_loss: 0.4056 - val_dept_output_accuracy: 0.6735 - val_dept_output_loss: 0.7527 - val_loss: 1.1593
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8319 - attrition_output_loss: 0.4124 - dept_output_accuracy: 0.6515 - dept_output_loss: 0.7472 - loss: 1.1582 - val_attrition_output_accuracy: 0.8605 - val_attrition_output_loss: 0.4006 - val_dept_output_accuracy: 0

In [ ]:
# Evaluate the model with the testing data
evaluate_da_model = da_model.evaluate(
    x_test_scaled, {'dept_output': test_y_encoded, 'attrition_output': y_test_encoded},
    verbose=1
)

print(evaluate_da_model)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8397 - attrition_output_loss: 0.5281 - dept_output_accuracy: 0.5810 - dept_output_loss: 0.8702 - loss: 1.3879 
[1.3439520597457886, 0.8445481061935425, 0.5569292306900024, 0.8367347121238708, 0.5918367505073547]


In [28]:
# Print the accuracy for both department and attrition
dept_prediction = da_model.predict(x_test_scaled)[0]
dept_accuracy = np.mean(np.argmax(dept_prediction, axis =1)== np.argmax(test_y_encoded, axis =1))

att_prediction = da_model.predict(x_test_scaled)[1]
att_accuracy = np.mean(np.argmax(att_prediction, axis =1)== np.argmax(y_test_encoded, axis =1))

print(f"Department Output Accuracy: {dept_accuracy}")
print(f"Attrition Output Accuracy: {att_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Department Output Accuracy: 0.5918367346938775
Attrition Output Accuracy: 0.8367346938775511


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I believe accuracy is not the best metric to use for this model, AUC-PR could be a better a beter metric to be used since it focuses on the imbalanced side of the data. Since most employees stay and a small percentage leave it makes it the positive class of the data. If the percent of attrtion was 10% AUC-PR would focus in on that versus the other data. This helps give insight to the false positives and false negatives 

2. For this function I used Softmax for the attrition and department output. In department output softmax allowed us to see the calssification of the deparmtent from the probabilities sum to 1. For attrition we used the probability to see the if it was either left the comapny(yes or no).

3. As previously mentioned, using AUC-PR to track attrition instead of accuracy will make the metrics more meaningful, as it better evaluates the model’s ability to identify employees who are likely to leave, especially in an imbalanced dataset. Increasing the number of epochs will allow the model to learn more complex patterns over time, while using larger batch sizes can lead to more stable updates during training. Additionally, balancing the classes will help prevent the model from being biased toward the majority class, improving its ability to correctly predict attrition cases and making the overall predictions more reliable.